In [518]:
from collections import defaultdict
from itertools import product
import numpy as np

---

In [872]:
def binvec (a, n):
    """take a binary vector a of length n in its decimal representation and return its binary represenation"""
    return format(a, '0'+str(n)+'b')

def vecadd (a, b, n, q=2):
    """take two vectors a and b of length n and return their sum (mod q)"""
    return ''.join([str((int(a[i]) + int(b[i])) % q) for i in range(n)])
    #return format(a^b, '0'+str(n)+'b') for binary vectors only
    
def vecsadd (vectors, n, q=2):
    """take a set of vectors as a list of strings of ints of length n
    and return their sum (mod q) as a string of ints"""
    
    # verify that all vectors are length n
    for vector in vectors:
        assert len(vector) == n, 'vectors are assumed to be of equal length'
        
    
    positions = defaultdict(list)
    for vector in vectors:
        for i in range(n ):
            positions[i].append(vector[i])
    summ = []
    for i, j in positions.items():
        summ.append(np.mod(np.array(j).astype(int).sum(), q))
        
    return ''.join([str(i) for i in summ])
    
def vecinv (a, n):
    return format(a^(2**n-1), '0'+str(n)+'b')

def v (n, q=2):
    """construct a linear space of dimension n over a finite field of order q"""
    return [''.join(str(pos) for pos in comb) for comb in list(product(range(q), repeat=n))]

def num_lin_sub_k (n, k, q):
    """Compute the number of k-dim linear subspaces of a linear space of dim n over a finite field of order q"""
    return np.prod([q**n-q**i for i in range(k)])/np.prod([q**k-q**i for i in range(k)])

def generate_scalar_multiples (vector, n, q):
    """Given a vector of length n over q symbols, get its scalar multiples"""
    scalar_multiples = []
    for scalar in range(q):
        scalar_multiple = vector
        for i in range(scalar):
            scalar_multiple = vecadd(scalar_multiple, vector, n, q)
        scalar_multiples.append(scalar_multiple)
    return scalar_multiples

def span (vectors=['1000','0100'], n=4, q=2):
    """
    get the span of a set of vectors in V(n, q) over GF(q)
    """
    if '0'*n in vectors:
        vectors.remove('0'*n)
    qsm = []
    for veck in vectors:
        qsm.append(generate_scalar_multiples(veck, n, q))

    return set([vecsadd(comb, n, q) for comb in product(*[qsm[i] for i in range(len(qsm))])])

In [171]:
"""
n = 8

a = binvec(31, n)
a_inv = vecinv(31, n)
b = binvec(64, n)

print(a)
print(a_inv)
print(b)
print(vecadd(a, b, n))
"""

00011111
11100000
01000000
01011111


---

# Construct $V(n, q)$ over $GF(q)$

$| V(n, q) | = q^n$

In [939]:
# specify the dimension of V(n, q)
n = 4
# specify the order of GF(q)
q = 3
# construct V(n, q) over GF(q)
space = v(n, q)
print('{}\t{}'.format('n',n))
print('{}\t{}'.format('q',q))
print('{}\t{}'.format('q^n',q**n))
print()
for i, c in enumerate(space): print('{}\t{}'.format(i, c))

n	4
q	3
q^n	81

0	0000
1	0001
2	0002
3	0010
4	0011
5	0012
6	0020
7	0021
8	0022
9	0100
10	0101
11	0102
12	0110
13	0111
14	0112
15	0120
16	0121
17	0122
18	0200
19	0201
20	0202
21	0210
22	0211
23	0212
24	0220
25	0221
26	0222
27	1000
28	1001
29	1002
30	1010
31	1011
32	1012
33	1020
34	1021
35	1022
36	1100
37	1101
38	1102
39	1110
40	1111
41	1112
42	1120
43	1121
44	1122
45	1200
46	1201
47	1202
48	1210
49	1211
50	1212
51	1220
52	1221
53	1222
54	2000
55	2001
56	2002
57	2010
58	2011
59	2012
60	2020
61	2021
62	2022
63	2100
64	2101
65	2102
66	2110
67	2111
68	2112
69	2120
70	2121
71	2122
72	2200
73	2201
74	2202
75	2210
76	2211
77	2212
78	2220
79	2221
80	2222


In [942]:
# show that the sum of any two vectors in the linear space is also in the linear space

a = np.random.choice(space) # choose a random vector in V(n, q)
b = np.random.choice(space) # choose a random vector in V(n, q)
c = vecadd(a, b, n, q)      # add them
print(a)
print(b)
print(c)
assert c in space

2002
2120
1122


---

## Compute the scalar multiples of a vector $\{λ\mathbf{v}$ | $\mathbf{v}\in V(n, q), \forall λ\in GF(q)\}$

In [974]:
# V(4, 3)
print('{}\t{}'.format(str(n**q) + ' vectors', str(q) + ' scalar multiples per vector'))
print()
for vector in space:
    print('{}\t{}'.format(vector, generate_scalar_multiples(vector, n, q)))

64 vectors	3 scalar multiples per vector

0000	['0000', '0000', '0000']
0001	['0001', '0002', '0000']
0002	['0002', '0001', '0000']
0010	['0010', '0020', '0000']
0011	['0011', '0022', '0000']
0012	['0012', '0021', '0000']
0020	['0020', '0010', '0000']
0021	['0021', '0012', '0000']
0022	['0022', '0011', '0000']
0100	['0100', '0200', '0000']
0101	['0101', '0202', '0000']
0102	['0102', '0201', '0000']
0110	['0110', '0220', '0000']
0111	['0111', '0222', '0000']
0112	['0112', '0221', '0000']
0120	['0120', '0210', '0000']
0121	['0121', '0212', '0000']
0122	['0122', '0211', '0000']
0200	['0200', '0100', '0000']
0201	['0201', '0102', '0000']
0202	['0202', '0101', '0000']
0210	['0210', '0120', '0000']
0211	['0211', '0122', '0000']
0212	['0212', '0121', '0000']
0220	['0220', '0110', '0000']
0221	['0221', '0112', '0000']
0222	['0222', '0111', '0000']
1000	['1000', '2000', '0000']
1001	['1001', '2002', '0000']
1002	['1002', '2001', '0000']
1010	['1010', '2020', '0000']
1011	['1011', '2022', '0000'

## Compute the sum $\mathbf{v_1} + ... + \mathbf{v_k}$ of a set of vectors $\{\mathbf{v_1}, ..., \mathbf{v_k}\} \in V(n, q)$

In [973]:
num_vecs = np.random.randint(1, 5)
sum_vecs = np.random.choice(space, size=num_vecs, replace=True)
summation = vecsadd(sum_vecs, n, q)

print('Summing {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(sum_vecs)))
print('Sum\t\t\t{}'.format(summation))

Summing 3 vectors	0202 + 0100 + 2001
Sum			2000


## Get the span of a set of vectors

In [1005]:
span_vecs = list(np.random.choice(space[1:], size=num_vecs, replace=False))
spn = span(span_vecs, n, q)

print('Spanning {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(span_vecs)))
print('Span {} vector{}\t\t{}'.format(len(spn), 's' if num_vecs != 1 else '', spn))

Spanning 3 vectors	2202 + 2021 + 0211
Span 9 vectors		{'2021', '0000', '1101', '0122', '1012', '0211', '1220', '2110', '2202'}


---

## Compute the number of distinct k-dim linear subspaces

The number of distinct linear subspaces of dim $k$ of a linear space of dim $n$ is $\frac{(q^n - q^0)(q^n - q^1)...(q^n - q^{k - 1})}{(q^k - q^0)(q^k - q^1)...(q^k - q^{k - 1})}$

In [1006]:
# Compute the total number of linear subspaces of a linear space of dim n
# i.e., compute the number of k-dim linear subspaces for k = 1, ..., n
# the "0-dim" linear subspace is the trivial subspace {0}

print('{}\t{}\t{}'.format('k', 'bases', 'n_choose_k'))
print()
for k in range(0, n + 1):
    print('{}\t{}\t{}'.format(k, int(num_lin_sub_k(n, k, q)), len(list(combinations(range(q**n-1), k)))))

k	bases	n_choose_k

0	1	1
1	40	80
2	130	3160
3	40	82160
4	1	1581580


---

# <font color="red">Under construction below this point</font>

## Get a basis for a k-dim linear subspace

In [392]:
k = 3

# this is just q^n - 1 choose k
bases = list(combinations(space[1:], k))
print(len(bases))
for i in bases:
    print(i)



def gen_dis_bases (space, k):
    bases = list(combinations(space[1:], k))

    distinct_bases = []
    while bases:
        basis, bases = bases[0], bases[1:]
        
        # check if the set of vectors is linearly independent
        elems = []
        for s in range(1, k + 1):
            elems += list(combinations(basis, s))
        if len(elems) < q**k - 1:
            continue
        
        lincoms = []
        pairs = list(combinations(basis, 2))
        for i, j in pairs:
            lincoms.append((i, j, vecadd(i, j, n)))
        
        for lincom in lincoms:
            if lincom in bases:
                bases.remove(lincom)
                
        distinct_bases.append(basis)
    return distinct_bases



distinct_bases = []
while bases:
    (b1, b2), bases = bases[0], bases[1:]
    b3 = vecadd(b1, b2, n)
    if (b1, b3) in bases:
        bases.remove((b1, b3))
    if (b2, b3) in bases:
        bases.remove((b2, b3))
    distinct_bases.append((b1, b2))
    
    
    
def gen_lin_code (basis):
    sums = [space[0]]
    sums += [i for i in basis]
    for i, j in list(combinations(basis, 2)):
        sums.append(vecadd(i, j, n))
    return sums

gen_lin_code(distinct_bases[0])